In [10]:
# A dictionary of movie critics and their ratings of a small
# set of movies
critics={
    
'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
 'The Night Listener': 3.0},

'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 3.5},

'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},

'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0,
 'You, Me and Dupree': 2.5},

'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0},

'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},

'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [11]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [12]:
critics['Toby']['Snakes on a Plane']=4.5
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

## Euclidean Distance Score

In [13]:
from math import sqrt
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
    for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

In [14]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

In [15]:
sim_distance(critics, 'Lisa Rose','Michael Phillips')

0.4444444444444444

In [16]:
sim_distance(critics, 'Lisa Rose','Claudia Puig')

0.2857142857142857

In [17]:
sim_distance(critics, 'Lisa Rose','Mick LaSalle')

0.3333333333333333

In [18]:
sim_distance(critics, 'Lisa Rose','Jack Matthews')

0.21052631578947367

In [19]:
sim_distance(critics, 'Lisa Rose','Toby')

0.2222222222222222

Lisa Rose is most similar to Gene Seymour

## Pearson Correlation Score

In [20]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
            
    # Find the number of elements
    n=len(si)
    
    # if they are no ratings in common, return 0
    if n==0: return 0
    
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r

In [21]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

In [22]:
sim_pearson(critics, 'Lisa Rose','Michael Phillips')

0.40451991747794525

In [23]:
sim_pearson(critics, 'Lisa Rose','Claudia Puig')

0.5669467095138396

In [24]:
sim_pearson(critics, 'Lisa Rose','Mick LaSalle')

0.5940885257860044

In [25]:
sim_pearson(critics, 'Lisa Rose','Jack Matthews')

0.7470178808339965

In [26]:
sim_pearson(critics, 'Lisa Rose','Toby')

0.9912407071619299

## Ranking The Critics

In [27]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [28]:
topMatches(critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

In [29]:
topMatches(critics,'Toby',3,sim_distance)

[(0.3076923076923077, 'Mick LaSalle'),
 (0.2857142857142857, 'Michael Phillips'),
 (0.23529411764705882, 'Claudia Puig')]

## Making Recommendations

In [30]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: continue
        sim=similarity(prefs,person,other)
 
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
 
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
    
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
 
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
 
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [31]:
getRecommendations(critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [32]:
getRecommendations(critics,'Toby', sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

## Matching items instead of users

In [33]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

In [34]:
movies=transformPrefs(critics)
topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [35]:
getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

# using the movie lens data set

In [42]:
def loadMovieLens(path='ml-100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item', encoding='latin-1'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [43]:
prefs = loadMovieLens( )

In [73]:
prefs['1'] # ratings of user 1

{'101 Dalmatians (1996)': 2.0,
 '12 Angry Men (1957)': 5.0,
 '20,000 Leagues Under the Sea (1954)': 3.0,
 '2001: A Space Odyssey (1968)': 4.0,
 'Abyss, The (1989)': 3.0,
 'Ace Ventura: Pet Detective (1994)': 3.0,
 'Air Bud (1997)': 1.0,
 'Akira (1988)': 4.0,
 'Aladdin (1992)': 4.0,
 'Alien (1979)': 5.0,
 'Aliens (1986)': 5.0,
 'All Dogs Go to Heaven 2 (1996)': 1.0,
 'Amadeus (1984)': 5.0,
 'Angels and Insects (1995)': 4.0,
 "Antonia's Line (1995)": 5.0,
 'Apocalypse Now (1979)': 3.0,
 'Apollo 13 (1995)': 4.0,
 'Aristocats, The (1970)': 2.0,
 'Army of Darkness (1993)': 4.0,
 'Austin Powers: International Man of Mystery (1997)': 4.0,
 'Babe (1995)': 1.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 2.0,
 'Basic Instinct (1992)': 3.0,
 'Batman & Robin (1997)': 1.0,
 'Batman Forever (1995)': 1.0,
 'Batman Returns (1992)': 1.0,
 'Beavis and Butt-head Do America (1996)': 3.0,
 'Bedknobs and Broomsticks (1971)': 2.0,
 'Belle de jour (1967)': 3.0,
 'Big Night (1996)': 5.0,
 'Billy Ma

In [74]:
getRecommendations(prefs,'1')[0:10] # get top 10 movies for user 1

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Little City (1998)'),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.999999999999999, 'Santa with Muscles (1996)')]

## Item-Based Collaborative Filtering

In [60]:
def calculateSimilarItems(prefs,n=10):
    # Create a dictionary of items showing which other items they are most similar to.
    result={}
    # Invert the preference matrix to be item-centric
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        # Status updates for large datasets
        c+=1
        if c%100==0: print (c,len(itemPrefs))
        # Find the most similar items to this one
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [63]:
itemsim = calculateSimilarItems(prefs,n=50)

100 1664
200 1664
300 1664
400 1664
500 1664
600 1664
700 1664
800 1664
900 1664
1000 1664
1100 1664
1200 1664
1300 1664
1400 1664
1500 1664
1600 1664


In [64]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    # Loop over items rated by this user
    for (item,rating) in userRatings.items( ):
        # Loop over items similar to this one
        for (similarity,item2) in itemMatch[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings: continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
    # Divide each total score by total weighting to get an average
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    # Return the rankings from highest to lowest
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [75]:
getRecommendedItems(prefs,itemsim,'1')[0:10] #recommendation for user 1

[(5.0, 'Winnie the Pooh and the Blustery Day (1968)'),
 (5.0, "What's Love Got to Do with It (1993)"),
 (5.0, 'Up Close and Personal (1996)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Speed (1994)'),
 (5.0, "Sophie's Choice (1982)"),
 (5.0, 'Something to Talk About (1995)'),
 (5.0, 'Some Like It Hot (1959)'),
 (5.0, 'Some Folks Call It a Sling Blade (1993)'),
 (5.0, 'Shine (1996)')]